In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import *

## **Read Dataset**

In [13]:
df = pd.read_csv('/kaggle/input/energy/energy_consumption.csv')
df.head()

,Datetime,DOM_MW
0,2005-12-31 01:00:00,9389.0
1,2005-12-31 02:00:00,9070.0
2,2005-12-31 03:00:00,9001.0
3,2005-12-31 04:00:00,9042.0
4,2005-12-31 05:00:00,9132.0


In [14]:
df = pd.read_csv('/kaggle/input/energy/energy_consumption.csv', index_col = 'Datetime', parse_dates = ['Datetime'])
df.head()

,DOM_MW
Datetime,
2005-12-31 01:00:00,9389.0
2005-12-31 02:00:00,9070.0
2005-12-31 03:00:00,9001.0
2005-12-31 04:00:00,9042.0
2005-12-31 05:00:00,9132.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 116189 entries, 2005-12-31 01:00:00 to 2018-01-02 00:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   DOM_MW  116189 non-null  float64
dtypes: float64(1)
memory usage: 1.8 MB


In [16]:
df.shape

(116189, 1)

## **Data Preprocessing**

In [17]:
scaler = MinMaxScaler()

In [18]:
scaled_df = scaler.fit_transform(df['DOM_MW'].values.reshape(-1,1))

In [21]:
scaled_df.shape

(116189, 1)

In [24]:
def Data_Preprocessing(data, time_step):
    x_train = []
    y_train = []
    
    for i in range(time_step, len(data)):
        x_train.append(data[i - time_step: i , 0])
        y_train.append(data[i, 0])
        
    x_test = x_train[110000: ]
    y_test = y_train[110000: ]
    
    x_train = x_train[: 110000]
    y_train = y_train[: 110000]
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    x_train = np.reshape(x_train, (110000, time_step , 1)) 
    x_test = np.reshape(x_test, (x_test.shape[0], time_step, 1))
    
    return (x_train, y_train, x_test, y_test)

In [25]:
time_step = 50
x_train, y_train, x_test, y_test = Data_Preprocessing(scaled_df, time_step)

In [26]:
x_train.shape

(110000, 50, 1)

In [27]:
y_train.shape

(110000,)

In [28]:
x_test.shape

(6139, 50, 1)

In [29]:
y_test.shape

(6139,)

## **RNN Model**

In [30]:
rnn_model = Sequential()

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True, input_shape = (x_train.shape[1], 1)))
rnn_model.add(Dropout(0.15))

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True))
rnn_model.add(Dropout(0.15))

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True))
rnn_model.add(Dropout(0.15))

rnn_model.add(Dense(1))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:
rnn_model.compile(optimizer = 'adam', loss = "MSE")
rnn_model.fit(x_train, y_train, epochs = 5)

Epoch 1/5
   7/3438 ━━━━━━━━━━━━━━━━━━━━ 1:14 22ms/step - loss: 0.4886

I0000 00:00:1711715413.779106      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3438/3438 ━━━━━━━━━━━━━━━━━━━━ 52s 13ms/step - loss: 0.0361
Epoch 2/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0141
Epoch 3/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0140
Epoch 4/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0139
Epoch 5/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 38s 11ms/step - loss: 0.0139


In [32]:
rnn_pred = rnn_model.predict(x_test)

192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step


## **LSTM Model**

In [33]:
lstm_model = Sequential()

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True, input_shape = (x_train.shape[1], 1)))
lstm_model.add(Dropout(0.15))

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True))
lstm_model.add(Dropout(0.15))

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True))
lstm_model.add(Dropout(0.15))

lstm_model.add(Dense(1))

In [34]:
lstm_model.compile(optimizer = 'adam', loss = "MSE")
lstm_model.fit(x_train, y_train, epochs = 5)

Epoch 1/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - loss: 0.0187
Epoch 2/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 31s 9ms/step - loss: 0.0140
Epoch 3/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 31s 9ms/step - loss: 0.0140
Epoch 4/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - loss: 0.0140
Epoch 5/5
3438/3438 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - loss: 0.0139
